In [1]:
%pip install torchvision
%pip install matplotlib
%pip install scipy
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
import os
import scipy
from torchvision import datasets, transforms
from torch import optim, nn, unsqueeze
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda, Compose
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split



In [3]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.Resize((28,28)), transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,))])

# with data augmentation for training only
train_transform = transforms.Compose([transforms.Resize((28,28)), transforms.ToTensor(),
                                    # transforms.ToPILImage(),
                                    # transforms.CenterCrop(21),
                                    transforms.RandomRotation(30),
                                    transforms.ColorJitter(brightness=0.2, contrast=0.2),
                                    # transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                                    transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
trainset0 = datasets.ImageFolder('/Users/enrico/Downloads/mnist-varres/train', transform=train_transform)
testset = datasets.ImageFolder('/Users/enrico/Downloads/mnist-varres/test', transform=transform)

# Also create a validation set 
trainset, valset = train_test_split(trainset0, test_size=10000, random_state=42)
#pytorch alternative for spliting into train and validation sets
#https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets

batch_size = 16

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)

In [4]:
training_data = enumerate(trainloader)
batch_idx, (images, labels) = next(training_data)
print(type(images)) # Checking the datatype 
print(images.shape) # the size of the image
print(labels.shape) # the size of the labels

<class 'torch.Tensor'>
torch.Size([16, 3, 28, 28])
torch.Size([16])


In [5]:
class Network(nn.Module):

    def __init__(self):
        super(Network, self).__init__()

        self.conv_neural_network_layers = nn.Sequential(
                # output_sizeOfEachConvLayer = [(in_channel + 2*padding - kernel_size) / stride] + 1

                # We have in_channels=1 because our input is a grayscale image
                nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1, stride=1), # (N, 1, 28, 28) 
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2), 

                # output of second conv layer
                nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1, stride=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2), 

                # output of third conv layer
                nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2) )

        # Linear layer
        self.linear_layers = nn.Sequential(
                nn.Linear(64*3*3, 10))
                # nn.Linear(16, 10)) # The output is 10 which should match the size of our class


    # Defining the forward pass 
    def forward(self, x):
        x = self.conv_neural_network_layers(x)
        # After we get the output of our convolutional layer we must flatten it or rearrange the output into a vector
        x = torch.flatten(x, 1) # same as x = x.view(x.size(0), -1)
        # Then pass it through the linear layer
        x = self.linear_layers(x)

        return x

model = Network()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


loss_fn = nn.BCEWithLogitsLoss() #nn.BCELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

#In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        y_hot = F.one_hot(y, 10)
        y_hot = torch.zeros(batch_size, 10)
        y_hot[range(y_hot.shape[0]), y]=1      

        X, y_hot = X.to(device), y_hot.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y_hot)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return loss

#We also check the model’s performance against the test dataset to ensure it is learning.

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0

    for batch, (X, y) in enumerate(dataloader):
        y_hot = F.one_hot(y, 10)
        y_hot = torch.zeros(batch_size, 10)
        y_hot[range(y_hot.shape[0]), y]=1      

        X, y_hot = X.to(device), y_hot.to(device)

        # Compute prediction error
        pred = model(X)
        test_loss += loss_fn(pred, y_hot).item()
        correct += (pred.argmax(axis=1) == y_hot.argmax(axis=1)).type(torch.float).sum().item()

    test_loss /= 10000
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss, 100*correct


## Experiments

In [6]:
epochs = 20

mmp_train_loss = []
mmp_loss = []
mmp_acc = []

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train(trainloader, model, loss_fn, optimizer)
    mmp_train_loss.append(train_loss)
    loss, acc = test(valloader, model, loss_fn)
    mmp_loss.append(loss)
    mmp_acc.append(acc)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 84.9%, Avg loss: 0.005715 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 89.7%, Avg loss: 0.004145 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 91.0%, Avg loss: 0.003587 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 91.2%, Avg loss: 0.003717 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 92.5%, Avg loss: 0.003309 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 92.7%, Avg loss: 0.003179 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.003098 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 93.4%, Avg loss: 0.003077 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 93.0%, Avg loss: 0.003149 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 93.5%, Avg loss: 0.002967 

Epoch 11
-------------------------------
Test Error: 
 Accuracy: 93.8%, Avg los